In [1]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from pyspark.sql.functions import col, expr

# Spark oturumu
builder = SparkSession.builder \
    .appName("Gold Layer - Forecasting") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

# CLTV tablosunu oku
df_cltv = spark.read.format("delta").load("../delta/gold/cltv_table")

# Basit bir tahmin modeli: ExpectedRevenue = CLTV * 12 (12 ay için varsayım)
df_forecast = df_cltv.withColumn("ExpectedRevenue_12M", col("CLTV") * expr("12"))

# Delta tablosu olarak kaydet
df_forecast.write.format("delta").mode("overwrite").save("../delta/gold/forecast_table")

df_forecast.select("CustomerID", "CLTV", "ExpectedRevenue_12M").show(5)

:: loading settings :: url = jar:file:/opt/miniconda3/envs/spark-delta-env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/alialtunoglu/.ivy2/cache
The jars for the packages stored in: /Users/alialtunoglu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-90153572-7dc0-4b9e-89dc-8b90f5113a41;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 76ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   

25/07/28 15:06:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/07/28 15:06:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/07/28 15:06:50 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/07/28 15:06:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+------------------+-------------------+
|CustomerID|              CLTV|ExpectedRevenue_12M|
+----------+------------------+-------------------+
|   15039.0| 2991148.887999999|3.589378665599999E7|
|   13259.0| 410.1909677419353|  4922.291612903224|
|   16982.0| 138.5134426229508| 1662.1613114754095|
|   17966.0|1965.6115789473686| 23587.338947368422|
|   13178.0| 56194.42777777777|  674333.1333333332|
+----------+------------------+-------------------+
only showing top 5 rows

